# Recognizing Flatlanders

## Data preprocessing

In [7]:
import h5py
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np

Read the training and test data from the HDF5 files.

In [18]:
with h5py.File('train_data.h5', 'r') as h5_file:
    x_train = np.array(h5_file['x_values'])
    y_train = np.array(h5_file['y_values'])
with h5py.File('test_data.h5', 'r') as h5_file:
    x_test = np.array(h5_file['x_values'])
    y_test = np.array(h5_file['y_values'])

Determine whether the backend expects the channel to come first or last in the input shape.

In [19]:
img_channels, img_rows, img_cols = 1, x_train.shape[1], x_train.shape[2]

if K.image_data_format() == 'channels_first':
    shape_ord = (img_channels, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, img_channels)

Reshape the data accordingly.

In [20]:
x_train = x_train.reshape((x_train.shape[0], ) + shape_ord)
x_test = x_test.reshape((x_test.shape[0], ) + shape_ord)

Convert the input data to single precision floating point, and scale between 0 and 1.

In [21]:
x_train = x_train.astype(np.float32)/255.0
x_test = x_test.astype(np.float32)/255.0

Verify input data format and shape.

In [22]:
print(x_train.shape, x_train.dtype, x_train.min(), x_train.max())
print(x_test.shape, x_test.dtype, x_test.min(), x_test.max())

(60000, 100, 100, 1) float32 0.0 1.0
(20000, 100, 100, 1) float32 0.0 1.0


Convert output to one-hot encoding.

In [23]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

Verify output data format and shape.

In [24]:
print(y_train.shape, y_train.dtype, y_train.min(), y_train.max())
print(y_test.shape, y_test.dtype, y_test.min(), y_test.max())

(60000, 3) float32 0.0 1.0
(20000, 3) float32 0.0 1.0


In [25]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)

In [26]:
print(x_train.shape, x_train.dtype, x_train.min(), x_train.max())
print(x_val.shape, x_val.dtype, x_val.min(), x_val.max())

(45000, 100, 100, 1) float32 0.0 1.0
(15000, 100, 100, 1) float32 0.0 1.0


In [27]:
print(y_train.shape, y_train.dtype, y_train.min(), y_train.max())
print(y_val.shape, y_val.dtype, y_val.min(), y_val.max())

(45000, 3) float32 0.0 1.0
(15000, 3) float32 0.0 1.0


## Model definition

In [28]:
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, LeakyReLU
from keras.layers.pooling import MaxPool2D
from keras.models import Sequential
from keras.optimizers import SGD, Adam

In [29]:
nr_filters = 32
nr_classes = y_train.shape[-1]
conv_x_size, conv_y_size = 5, 5
model = Sequential()
model.add(Conv2D(nr_filters, (conv_x_size, conv_y_size), strides=2,
                 padding='valid', input_shape=shape_ord))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Conv2D(nr_filters*2, (conv_x_size, conv_y_size), strides=2,
                 padding='valid'))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Conv2D(nr_filters*4, (conv_x_size, conv_y_size), strides=2,
                 padding='valid'))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(nr_classes))
model.add(Activation('softmax'))

In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 48, 48, 32)        832       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 64)        51264     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 9, 9, 128)         204928    
__________

In [31]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(),
              metrics=['accuracy'])

In [32]:
history = model.fit(x_train, y_train, epochs=20, batch_size=64,
                    validation_data=(x_val, y_val))

Train on 45000 samples, validate on 15000 samples
Epoch 1/20
45000/45000 [==============================] - 10s 219us/step - loss: 0.9788 - acc: 0.4686 - val_loss: 0.9355 - val_acc: 0.5038
Epoch 2/20
45000/45000 [==============================] - 8s 172us/step - loss: 0.9351 - acc: 0.5060 - val_loss: 1.1716 - val_acc: 0.4183
Epoch 3/20
45000/45000 [==============================] - 8s 177us/step - loss: 0.9280 - acc: 0.5114 - val_loss: 0.9229 - val_acc: 0.5153
Epoch 4/20
45000/45000 [==============================] - 8s 179us/step - loss: 0.9220 - acc: 0.5160 - val_loss: 0.9267 - val_acc: 0.5119
Epoch 5/20
45000/45000 [==============================] - 8s 173us/step - loss: 0.9145 - acc: 0.5216 - val_loss: 0.9055 - val_acc: 0.5294
Epoch 6/20
45000/45000 [==============================] - 8s 174us/step - loss: 0.9046 - acc: 0.5317 - val_loss: 0.9041 - val_acc: 0.5403
Epoch 7/20
45000/45000 [==============================] - 8s 173us/step - loss: 0.8888 - acc: 0.5452 - val_loss: 0.8758 -

In [33]:
model.evaluate(x_test, y_test)

20000/20000 [==============================] - 1s 74us/step


[0.006571922181593254, 0.99905]